In [6]:
import mmf_setup;mmf_setup.nbinit(hgroot=False)
from mmfutils.contexts import NoInterrupt

<IPython.core.display.Javascript object>

# Understanding 2D BdG Numerical Simulation
I find it's not easy to make a smooth translation from abstract knowledge of quantum theory to 2D numerical code, it's good to decompose all details first, and make connection to every line of code.

start with schodinger equation for 2D system, it should look like:
$$
H(x,y,t)\psi(x,y)=E\psi(x,y)
$$

the wavefunction in a 2D grid can be presented as a 2D matrix
$$\psi =
\begin{pmatrix}
    \psi_{1,1} & \psi_{1,2}&...&\psi_{1,N}\\
    \psi_{2,1} & \psi_{2,1}&...&\psi_{1,N}\\
    \vdots\\
    e^\psi_{N,1} & \psi_{N,2}&...&\psi_{N,N}\\
\end{pmatrix}
$$

Then, for any operator, it's matrix element form can be defined as:

\begin{align}
O_{m,n}&=\braket{\psi_m |\hat{O}|\psi_n}\\
&= \int dx dy {}\psi_{m}^*(x,y)\hat{O}\psi_n(x,y) \\
&=\Delta_x\Delta_y\sum_{x,y} ???
\end{align}

Depends on what the operator, if $\hat{O}$ is potential operator depends only on spatial coordinate $x,y$, we can easily have: 
\begin{align}
O_{m,n}&=\braket{\psi_m |\hat{O}|\psi_n}\\
&= \int dx dy {}\psi_m(x,y)\hat{O}(x,y)\psi_n(x,y) \\
&=\Delta_x\Delta_y\sum_{x,y} \psi_m^*(x,y)\hat{O}(x,y)\psi_n(x,y) \\
&=\Delta_x\Delta_y (\psi_{1,1}^* O_{1,1}\psi_{1,1}+ \psi_{1,2}^* O_{1,2}\psi_{1,2} +...+\psi_{2,1}^* O_{2,1}\psi_{2,1}+ \psi_{2,1}^* O_{2,1}\psi_{2,1} +...+...) \\
&=\Delta_x\Delta_y (\abs{\psi_{1,1}}^2 O_{1,1}+ \abs{\psi_{1,2}}^2 O_{1,2} +...+\abs{\psi_{2,1}}^2 O_{2,1}+ \abs{\psi_{2,1}}^2 O_{2,1} +...+...) \\
&=O(x,y)
\end{align}
where $\Delta_x, \Delta_y$ are the lattice spacings, and operators that depends only on spacial freedoms are trial, as we can see we just need to descetize them in lattice. **What's a fool I am to go through such long way to find it, LOL!**

### Fourier integral to Fourier series

Similar as the 1D FFT
$$
  FFT[\psi(x)] = \int\d{x}\; e^{-\I k x}\psi(x) = \frac{L}{N}\sum_{n} e^{-\I k x_n}\psi(x_n),\\
  FFT^{-1}(\psi_k) = \int\frac{\d{k}}{(2\pi)}\; e^{\I k x}\psi_k = \frac{1}{L}\sum_{m} e^{\I k_m x}\psi_{k_m}.
$$
we can extend these relations to 2D case without difficulty:
$$
  FFT[\psi(x,y)] = \int dx dy e^{-i(k_x x +k_y y)}\psi(x) = \frac{L_x L_y}{N_x N_y}\sum_{m,n} e^{-i (k_m x_m + k_n y_n)}\psi(x_n,y_n)\tag{1},\\
$$
$$
  FFT^{-1}[\psi(k_x,k_y)] = \int\frac{dk_x dk_y}{(4\pi^2)}\; e^{i (k_x x + k_y y)}\psi(k_x, k_y) = \frac{1}{L_x L_y}\sum_{m,n} e^{i (k_m x_m + k_n y_n)}\psi(k_m,k_n) \tag{2}
$$

For operators like kinetic operator, we may need to use Fourier transform to change the basis and backward. The kinetic term for 2D system is:

$$
\hat{T}=\frac{-\hbar^2}{2m}\left[\frac{\partial^2 }{\partial x^2}+\frac{\partial^2 }{\partial y^2}\right]\tag{3}
$$
when it acts on an wavefunction $\psi(x,y)$, yields another function $\phi(x,y)$
$$
\phi(x,y)=\hat{T}\psi(x,y)=\frac{-\hbar^2}{2m}\left[\frac{\partial^2 }{\partial x^2}+\frac{\partial^2 }{\partial y^2}\right]\psi(x,y)\tag{4}
$$
To make is more obviouse for computationn purpose, we use Fourier transform:

$$
\psi(x,y)=\frac{1}{2\pi}\int dk_x dk_y\psi(k_x,k_y)e^{-i(k_xx+k_yy)}\tag{5}
$$
where $\psi(k_x,k_y)$ is just Fourier transfrom of $\psi$, in mometum space.
$$
\psi(k_x,k_y)=\frac{1}{2\pi} \int dx dy \psi(x,y)e^{i(k_xx+k_yy)}\tag{6}
$$

If $\psi(x,y)$ is represented as (1), we can elimilate the second order partial differitial to get:

\begin{align}
\phi(x,y)&=\hat{T}\psi(x,y)\\
&=\frac{-\hbar^2}{2m}\left[\frac{\partial^2 }{\partial x^2}+\frac{\partial^2 }{\partial y^2}\right]\psi(x,y)\\
&=\frac{\hbar^2}{2m}\frac{1}{2\pi} \int dk_x dk_y \left[k_x^2 +k_y^2 \right]\psi(k_x,k_y)e^{-i(k_xx+k_yy)}\\
&=\frac{\hbar^2}{2m}\frac{1}{4\pi^2} \int dk_x dk_y dx' dy' \left[k_x^2 +k_y^2 \right]\psi(x',y')e^{-i(k_xx+k_yy-k_xx'- k_yy')}\tag{7}
\end{align}

Transfrom from integral to summation yiels:
$$
\phi(x,y) = \frac{\hbar^2}{2m}\frac{1}{4\pi^2} \frac{1}{N_xN_y} \sum_{k,l,m,n}\left[k_{x_k}^2+k_{y_l}^2\right]\psi(x'_m,y'_n)e^{-i\left[k_{x_k}(x-x'_m)+k_{y_l}(y-y'_n)\right]}\\
=\sum_{k_x,k_y,m,n}f(k_x,k_y)\psi(x'_m,y'_n)e^{-i\left[k_x(x-x'_m)+k_y(y-y'_n)\right]}\tag{8}
$$
where

$$
f(k_x,k_y)=\frac{\hbar^2}{2m}\frac{1}{4\pi^2} \frac{1}{N_xN_y} (k_x^2+k_y^2)
$$



Now, to present on matrix or tensor form, we need to represent the LHS as a matrix:

\begin{align}
\phi(x,y)
&=\sum_{k_x,k_y}f(k_x,k_y)e^{-i(k_xx+k_yy)}\left\{
\left[\psi(x'_1,y'_1)e^{i(k_xx'_1+k_yy'_1)} + \psi(x'_1,y'_2)e^{i(k_xx'_1+k_yy'_2)} +...+\psi(x'_1,y'_{N_y})e^{i(k_xx'_1+k_yy'_{N_y})}\right] +\\
\left[\psi(x'_2,y'_1)e^{i(k_xx'_2+k_yy'_1)} + \psi(x'_2,y'_2)e^{i(k_xx'_2+k_yy'_2)} +...+\psi(x'_2,y'_{N_y})e^{i(k_xx'_2+k_yy'_{N_y})}\right] +\\
\vdots \\
\left[\psi(x'_{N_x},y'_1)e^{i(k_xx'_{N_x}+k_yy'_1)} + \psi(x'_{N_x},y'_{N_x})e^{i(k_xx'_{N_x}+k_yy'_2)} +...+\psi(x'_{N_x},y'_{N_y})e^{i(k_xx'_{N_x}+k_yy'_{N_y})}\right]
\right\} \tag{9}
\end{align}

The braket terms can be rearragned as:
$$
e^{ik_xx'_1}\left[\psi(x'_1,y'_1)e^{ik_yy'_1} + \psi(x'_1,y'_2)e^{ik_yy'_2}+...\psi(x'_1,y'_{N_y})e^{ik_yy'_{N_y}}\right] +\\
e^{ik_xx'_2}\left[\psi(x'_2,y'_1)e^{ik_yy'_1} + \psi(x'_2,y'_2)e^{ik_yy'_2}+...\psi(x'_2,y'_{N_y})e^{ik_yy'_{N_y}}\right] +\\
\vdots\\
e^{ik_xx'_{x_N}}\left[\psi(x'_{x_N},y'_1)e^{ik_yy'_1} + \psi(x'_{x_N},y'_2)e^{ik_yy'_2}+...\psi(x'_{x_N},y'_{N_y})e^{ik_yy'_{N_y}}\right] \tag{10}
$$

Careful obervation it can be found we can put this into matrix form, and define it as:
$$
  M(k_x,k_y)=\begin{pmatrix}
    e^{ik_xx'_1} & e^{ik_xx'_2} & ... e^{ik_xx'_{N_x}}
  \end{pmatrix} 
  \begin{pmatrix}
    \psi(x'_1,y'_1) & \psi(x'_1,y'_2)&...&\psi(x'_1,y'_{N_y})\\
    \psi(x'_2,y'_1) & \psi(x'_2,y'_2)&...&\psi(x'_2,y'_{N_y})\\
    \vdots\\
    \psi(x'_{N_x},y'_1) & \psi(x'_{N_x},y'_2)&...&\psi(x'_{N_x},y'_{N_y})\\
  \end{pmatrix}
 \begin{pmatrix}
    e^{ik_yy'_1}\\
    e^{ik_yy'_2}\\
    \vdots\\
    e^{ik_yy'_{N_y}}\\
  \end{pmatrix}.\tag{11}
$$

Then $\phi(x,y)$ can be put as:
\begin{align}
\phi(x,y) 
&= \sum_{k_x,k_y} f(k_x,k_y)M(k_x,k_y)e^{-i(k_xx+k_yy)} \\
&=
\begin{pmatrix}
    e^{-ik_{x_1}x} & e^{-ik_{x_2}x} & ... e^{-ik_{x_{N_x}}x}
  \end{pmatrix} 
  \begin{pmatrix}
     Mf(k_{x_1},k_{y_1}) & Mf(k_{x_1},k_{y_2}&...&Mf(k_{x_1},k_{y_{N_y}})\\
    Mf(k_{x_2},k_{y_1}) & Mf(k_{x_2},k_{y_2}&...&Mf(k_{x_2},k_{y_{N_y}})\\
    \vdots\\
    Mf(k_{x_{N_x}},k_{y_1}) & Mf(k_{x_{N_x}},k_{y_2}&...&Mf(k_{x_{N_x}},k_{y_{N_y}})\\
  \end{pmatrix}
 \begin{pmatrix}
    e^{-ik_{y_1}y}\\
    e^{-ik_{y_2}y}\\
    \vdots\\
    e^{-ik_{y_{N_y}}y}\\
  \end{pmatrix}.\tag{12}
\end{align}


where $Mf$ is a matrix eqaul to the elementwise product of two matrix with the same dimension $N_x\times N_y$.

if the indics $x,y$ run over all values, we can get the $\phi(x,y)$ in grid:
$$
[\phi]=U_x^T Mf U_y
$$
where
$$
U_x = 
\begin{pmatrix}
    e^{-ik_{x_1}x_1}&  e^{-ik_{x_1}x_2}&...&e^{-ik_{x_1}x_{N_x}}\\
    e^{-ik_{x_2}x_1}&  e^{-ik_{x_2}x_2}&...&e^{-ik_{x_2}x_{N_x}}\\
    \vdots\\
    e^{-ik_{x_{N_x}}x_1}&  e^{-ik_{x_{N_x}}x_2}&...&e^{-ik_{x_{N_x}}x_{N_x}}\\
  \end{pmatrix}\tag{13}
  $$
  
  $$
U_y = 
\begin{pmatrix}
    e^{-ik_{y_1}y_1}&  e^{-ik_{y_1}y_2}&...&e^{-ik_{y_1}y_{N_y}}\\
    e^{-ik_{y_2}y_1}&  e^{-ik_{y_2}y_2}&...&e^{-ik_{y_2}y_{N_y}}\\
    \vdots\\
    e^{-ik_{y_{N_y}}x_1}&  e^{-ik_{y_{N_x}}x_2}&...&e^{-ik_{y_{N_y}}y_{N_y}}\\
  \end{pmatrix} \tag{14}
$$
and 
$$
[f]=\frac{\hbar^2}{2m}\frac{1}{N_xN_y}
\begin{pmatrix}
    k_{x_1}^2 +  k_{y_1}^2&  k_{x_1}^2 +  k_{y_2}^2&...& k_{x_1}^2 +  k_{y_{N_y}}^2\\
    k_{x_2}^2 +  k_{y_1}^2&  k_{x_2}^2 +  k_{y_2}^2&...& k_{x_2}^2 +  k_{y_{N_y}}^2\\
    \vdots\\
    k_{x_{N_x}}^2 +  k_{y_1}^2&  k_{x_{N_x}}^2 +  k_{y_2}^2&...& k_{x_{N_x}}^2 +  k_{y_{N_y}}^2\\
  \end{pmatrix} \tag{15}
$$

and 
$$
[M] = U_x^* [\psi] U_y^\dagger
$$

Since $Mf$ is the elementwise product of two matrix, we can factor out the $[\psi]$

$$
Mf= [f] U_x^*[\psi] U_y^\dagger = U_x^*[f]U_y^\dagger [\psi] \tag{16}
$$

Then the $[\phi]$ can be put as:

$$
[\phi] = U_x^T U_x^*[f]U_y^\dagger U_y [\psi] \tag{17}
$$

Or the kinectic matrix can be written:
$$
[\hat{T}] = U_x^T U_x^*[f]U_y^\dagger U_y \tag{18}
$$

<font color='red'>**I got a feeling that something goes wrong with above derivation, the T matrix and wavefunction vector should satisfy the matrix dot vector convention.**</font>

Let's go back to equation (10)

$$
e^{ik_xx'_1}\left[\psi(x'_1,y'_1)e^{ik_yy'_1} + \psi(x'_1,y'_2)e^{ik_yy'_2}+...\psi(x'_1,y'_{N_y})e^{ik_yy'_{N_y}}\right] +\\
e^{ik_xx'_2}\left[\psi(x'_2,y'_1)e^{ik_yy'_1} + \psi(x'_2,y'_2)e^{ik_yy'_2}+...\psi(x'_2,y'_{N_y})e^{ik_yy'_{N_y}}\right] +\\
\vdots\\
e^{ik_xx'_{x_N}}\left[\psi(x'_{x_N},y'_1)e^{ik_yy'_1} + \psi(x'_{x_N},y'_2)e^{ik_yy'_2}+...\psi(x'_{x_N},y'_{N_y})e^{ik_yy'_{N_y}}\right] \tag{10}
$$

if we treat $e^{i(k_xx'_m + k_yy'_n}$ as a vector of size $N_x \times N_y$ and $\psi$ also with sime size 1d vector, then (10) can be put as:
\begin{align}
M(k_x,k_y)\psi&=
 \begin{pmatrix}
    e^{i(k_xx_1+k_yy_1)}& e^{i(k_xx_1+k_yy_2)}&...& e^{i(k_xx_1+k_yy_{N_y})}& e^{i(k_xx_2+k_yy_1)}&&...&e^{i(k_xx_2+k_yy_{N_y})}&...& e^{-i(k_xx_{N_x}+k_yy_{N_y})}
  \end{pmatrix}
  \begin{pmatrix}
    \psi_{1,1}\\
    \psi_{1,2} \\
    \vdots\\
    \psi_{1,N_y} \\
    \psi_{2,1}\\
    \vdots\\
    \psi_{2,N_y}\\
    \vdots\\
    \psi_{N_x,N_y}
  \end{pmatrix}\\
   &=\sum_{n}^{N}U(k_x,k_y)_n \psi_n  \tag{19}
 \end{align}

Define

$$
M'(k_x,k_y)=\sum_{n}^{N}U(k_x,k_y)_n f_n 
$$

where $N=N_x\times N_y$

  
  The equation (12) need to be modified too:

\begin{align}
\phi(x,y) 
&= \sum_{k_x,k_y} M'(k_x,k_y)e^{-i(k_xx+k_yy)} \\
&= \begin{pmatrix}
    e^{-i(k_{x_1}x+k_{x_1}y)}& e^{-i(k_{x_1}x+k_{y_2}y)}&...&e^{-i(k_{x_1}x+k_{y_{N_y}}y)}& e^{-i(k_{x_2}x+k_{y_1}y)}&&...&e^{-i(k_{x_2}x+k_{y_{N_y}}y)}&...& e^{-i(k_{x_{N_x}}x+k_{y_{N_y}}y)}
  \end{pmatrix}
  \begin{pmatrix}
    M'_{1,1}\\
    M'_{1,2} \\
    \vdots\\
    M'_{1,N_y} \\
    M'_{2,1}\\
    \vdots\\
    M'_{2,N_y}\\
    \vdots\\
    M'_{N_x,N_y}
  \end{pmatrix}\\
   &=\begin{pmatrix}
    e^{-i(k_{x_1}x+k_{x_1}y)}& e^{-i(k_{x_1}x+k_{y_2}y)}&...&e^{-i(k_{x_1}x+k_{y_{N_y}}y)}& e^{-i(k_{x_2}x+k_{y_1}y)}&&...&e^{-i(k_{x_2}x+k_{y_{N_y}}y)}&...& e^{-i(k_{x_{N_x}}x+k_{y_{N_y}}y)}
  \end{pmatrix} M'
 \end{align} 

Now it's much clear, if we run $x, y$ over all pair of values, we get:
$$
[\phi]= \hat{U} M'\psi
$$

where 
$$
\hat{U} 
=\begin{pmatrix}
    e^{-i(k_{x_1}x_1+k_{y_1}y_1)}& e^{-i(k_{x_1}x_1+k_{y_2}y_1)}&...&e^{-i(k_{x_1}x_1+k_{y_{N_y}}y_1)}& e^{-i(k_{x_2}x_1+k_{y_1}y_1)}&&...&e^{-i(k_{x_2}x_1+k_{y_{N_y}}y_1)}&...& e^{-i(k_{x_{N_x}}x_1+k_{y_{N_y}}y_1)} \\
    e^{-i(k_{x_1}x_1+k_{y_1}y_2)}& e^{-i(k_{x_1}x_1+k_{y_2}y_2)}&...&e^{-i(k_{x_1}x_1+k_{y_{N_y}}y_2)}& e^{-i(k_{x_2}x_1+k_{y_1}y_2)}&&...&e^{-i(k_{x_2}x_1+k_{y_{N_y}}y_2)}&...& e^{-i(k_{x_{N_x}}x_1+k_{y_{N_y}}y_2)} \\
    \vdots \\
   e^{-i(k_{x_1}x_{N_x}+k_{y_1}y_{N_y})}& e^{-i(k_{x_1}x_{N_x}+k_{y_2}y_{N_y})}&...&e^{-i(k_{x_1}x_{N_x}+k_{y_{N_y}}y_{N_y})}& e^{-i(k_{x_2}x_{N_x}+k_{y_1}y_{N_y})}&&...&e^{-i(k_{x_2}x_{N_x}+k_{y_{N_y}}y_{N_y})}&...& e^{-i(k_{x_{N_x}}x_{N_x}+k_{y_{N_y}}y_{N_y})} \\
  \end{pmatrix}
$$


A bit careful examination, it's not hard to find:
$$
\phi=\hat{U}M'=\hat{U}f\hat{U}^\dagger \psi
$$

which means:
$$
[\hat{T}] = \hat{U}f\hat{U}^\dagger \tag{20}
$$

Now, result (20) seems to make sense, it may be used to check if it agree with Dr. Forbes' code:

In [9]:
from __future__ import division
import itertools
import numpy as np
import scipy.integrate
from mmfutils.math.integrate import mquad

class BCS(object):
    hbar = 1.0
    m = 1.0
    def __init__(self, Nxy=(32, 32), Lxy=(10.0, 10.0), dx=None, T=0,E_c=np.inf):
      
        dy = dx
        if dx is None:
            dx, dy = np.divide(Lxy, Nxy)
        elif Lxy is None:
            Lxy = np.prod(Nxy, dx)
        elif Nxy is None:
            Nxy = np.ceil(Lxy / dx).astype(int)

        Nx, Ny = Nxy
        Lx, Ly = Lxy
        self.xy = ((np.arange(Nx) * dx - Lx / 2)[:, None],# half of the length
                   (np.arange(Ny) * dy - Ly / 2)[None, :])
        
        self.kxy = (2*np.pi * np.fft.fftfreq(Nx, dx)[:, None],
                    2*np.pi * np.fft.fftfreq(Ny, dy)[None, :])
        self.dx = dx
        
        self.Nxy = tuple(Nxy)
        self.Lxy = tuple(Lxy)
        self.T = T
        self.E_c = E_c
        
    def get_Ks(self, twist=(0, 0)):
    
        mat_shape = (np.prod(self.Nxy),)*2
        tensor_shape = self.Nxy + self.Nxy
        K = np.eye(mat_shape[0]).reshape(tensor_shape)
        K = self.hbar**2/2/self.m *self.ifftn(sum(_k**2 for _k in self.kxy)[:, :,  None, None]*self.fftn(K)).reshape((np.prod(self.Nxy),)*2).reshape(mat_shape)
        return K

    def fftn(self, y):
            return np.fft.fftn(y, axes=(0,1))
            
    def ifftn(self, y):
            return np.fft.ifftn(y, axes=(0,1))
            
   

In [12]:
# The code is a bit silly, but it works, and it's connected to all what I have done above
# Obviously can be shorten a lot 
def getKineticMatrix(s):
    U = []
    for x in s.xy[0].ravel():
        for y in s.xy[0].ravel():
            for kx in s.kxy[0].ravel():
                for ky in s.kxy[1].ravel():
                    U.append(np.exp(-1j*(kx * x + ky * y)) )

    a= np.reshape(U,(np.prod(s.Nxy),)*2 )

    f = []
    for k1 in s.kxy[0].ravel():
        for k2 in s.kxy[1].ravel():
            f.append((k1**2 + k2**2 )/2)
    b = np.diag(f) / np.prod(s.Nxy)
    T=np.matmul(a,b)
    T2 = np.matmul(T,a.conj().T)
    return T2

In [13]:
s = BCS(Nxy=(2**1,)*2)
T1 = s.get_Ks()
s.Nxy
T2 = getKineticMatrix(s)
assert np.allclose(T1, T2)
T1, T2

(array([[ 0.19739209+0.j, -0.09869604+0.j, -0.09869604+0.j,
          0.        +0.j],
        [-0.09869604+0.j,  0.19739209+0.j,  0.        +0.j,
         -0.09869604+0.j],
        [-0.09869604+0.j,  0.        +0.j,  0.19739209+0.j,
         -0.09869604+0.j],
        [ 0.        +0.j, -0.09869604+0.j, -0.09869604+0.j,
          0.19739209+0.j]]),
 array([[ 1.97392088e-01+0.00000000e+00j, -9.86960440e-02-1.81301692e-17j,
         -9.86960440e-02-1.81301692e-17j,  0.00000000e+00+1.20867794e-17j],
        [-9.86960440e-02+1.81301692e-17j,  1.97392088e-01+0.00000000e+00j,
          1.48020358e-33+0.00000000e+00j, -9.86960440e-02-1.81301692e-17j],
        [-9.86960440e-02+1.81301692e-17j,  1.48020358e-33+0.00000000e+00j,
          1.97392088e-01+0.00000000e+00j, -9.86960440e-02-1.81301692e-17j],
        [ 0.00000000e+00-1.20867794e-17j, -9.86960440e-02+1.81301692e-17j,
         -9.86960440e-02+1.81301692e-17j,  1.97392088e-01+0.00000000e+00j]]))

In [5]:
for n in range(1,5):
    s = BCS(Nxy=(2**n,)*2)
    T1 = s.get_Ks()
    s.Nxy
    T2 = getKineticMatrix(s)
    assert np.allclose(T1, T2)

They agree, cheers!

# Understanding ASLDA
## Modified kinetic density matrix
In the ASLDA, the kinetic density matrix changed, unlike in homogenous case above, which is just second differential operators, we need to derive it form from mathmetical form:

$$
\begin{pmatrix}
K_a - \mu_a + V_a & \Delta^\dagger \\
\Delta & -K_b+\mu_b - V_b\\
\end{pmatrix}
\begin{pmatrix}
u_n\\
v_n
\end{pmatrix}=E_n
\begin{pmatrix}
u_n\\
v_n
\end{pmatrix}
$$

where the $K_a, K_b$ is defined:

\begin{align}
K_au
&=-\frac{\hbar^2}{2m}\nabla(\alpha_a(n_a,n_b))\nabla u)\\
&=-\frac{\hbar^2}{2m}\left[\nabla\alpha_a\nabla + \alpha\nabla^2\right]u \tag{21}
\end{align}

$$
\nabla = i\frac{\partial}{\partial x} + j \frac{\partial}{\partial y}
$$


What we need to do is to put (21) into a matrix representation, we can use the same reasoning as we did in the last section, and the result can be gotten:
\begin{align}
\nabla \alpha_a \nabla + \alpha\nabla^2 
&=Diag(\hat{\nabla}\alpha)\hat{\nabla} +  Diag(\alpha)\hat{U}f\hat{U}^\dagger
&=Diag(\hat{\nabla}\alpha)\hat{\nabla} +  Diag(\alpha)\hat{T}
\end{align}
where 
$$
\hat{\nabla} =\hat{U}g\hat{U}^\dagger
$$
and
$$
g(k_a,k_b) =  -i(k_a + k_b)\\
f(k_a,k_b) = -(k_a^2 + k_b^2)
$$

## Potential terms
The potential terms is not totally different from the simple external potentials, we need to incorporate lots of tems into the modified potentials, as can be seen from their expressions:

$$
V_a=\frac{\partial\alpha_-(n_a,n_b)}{\partial n_a}\frac{\hbar^2\tau_-}{2m} + \frac{\partial \alpha_+(n_a,n_b)}{\partial n_a}\left(\frac{\hbar^2\tau_+}{2m} - \frac{\Delta^\dagger\upsilon}{\alpha_+(n_a,n_b)}\right)-\frac{\partial\tilde{C}(n_a,n_b)}{\partial n_a}\frac{\Delta^\dagger \Delta}{\alpha_+} + \frac{\hbar^2}{m} \frac{\partial D(n_a,n_b)}{\partial n_a}+U_a(r)
$$
where

\begin{align}
n_a(r)&=\sum_{\abs{E_n}<E_c} \abs{u_n(r)}^2 f_\beta(E_n)\\
n_b(r)&=\sum_{\abs{E_n}<E_c} \abs{v_n(r)}^2 f_\beta(-E_n)\\
\tau_a(r)&=\sum_{\abs{E_n}<E_c} \abs{\nabla u_n(r)}^2 f_\beta(E_n)\\
\tau_b(r)&=\sum_{\abs{E_n}<E_c} \abs{\nabla v_n(r)}^2 f_\beta(-E_n)\\
p(r)&=\frac{n_a(r)-n_b(r)}{n_a(r)+n_b(r)}\\
\alpha(p)&=1.094+0.156p \left(1-\frac{2p^2}{3}+\frac{p^4}{5}\right)-0.532p^2\left(1-p^2+\frac{p^4}{3} \right)\\
\alpha_a(n_a,n_b)&=\alpha(p)\\
\alpha_b(n_a,n_b)&=\alpha(-p)\\
\alpha_{\pm}(n_a,n_b)&=\frac{1}{2}\left[\alpha_a(n_a,n_b)\pm\alpha_b(n_a,n_b)\right]\\
\tau_{\pm}&=\tau_a \pm \tau_b\\
G(p)&=0.357+0.642p^2\\
\tilde{C}(n_a,n_b)&=\frac{\alpha_+(p)(n_a+n_b)^{1/3}}{\gamma(p)}\\
D(n_a,n_b)&=\frac{\left(6\pi^2(n_a+n_b)\right)^{5/3}}{20\pi^2}\left[G(p)-\alpha(p)\left(\frac{1+p}{2}\right)^{5/3}-\alpha(-p)\left(\frac{1-p}{2}\right)^{5/3}\right]
\end{align}

The partial derivatives then can be calculated:
\begin{align}
\frac{\partial \alpha_-}{\partial n_a}&=\frac{1}{2}\\
\frac{\partial \alpha_-}{\partial n_b}&=-\frac{1}{2}\\
\frac{\partial p}{\partial n_a}&=\frac{2 n_b}{\left(n_a+n_b\right)^2}\\
\frac{\partial p}{\partial n_b}&=-\frac{2 n_a}{\left(n_a+n_b\right)^2}\\
\frac{d\alpha}{dp}
&=0.156 \left(\frac{4 p^3}{5}+\frac{4 p}{3}\right) p-1.064 \left(\frac{p^4}{3}-p^2+1\right) p+0.156 \left(\frac{p^4}{5}+\frac{2 p^2}{3}+1\right)-0.532 \left(\frac{4 p^3}{3}-2 p\right) p^2\\
&=-1.064 p^5+0.156 p^4+2.128 p^3+0.312 p^2-1.064 p+0.156\\
\frac{d \alpha_+}{d p}&=-1.064 p^5+2.128 p^3-1.064 p\\
\frac{d \alpha_-}{d p}&=0.156 \left(p^2+1.\right)^2\\
\end{align}

### $\frac{\partial \alpha}{\partial n}$ Terms:

\begin{align}
\frac{\partial \alpha_+(n_a,n_b)}{\partial n_a} &= \frac{n_a^2 n_b^3 \left(34.048 n_b-34.048 n_a\right)}{\left(n_a+n_b\right){}^7}\\
\frac{\partial \alpha_+(n_a,n_b)}{\partial n_b} &= \frac{n_a^3 n_b^2 \left(34.048 n_a-34.048 n_b\right)}{\left(n_a+n_b\right){}^7}\\
\frac{\partial \alpha_-(n_a,n_b)}{\partial n_a} &= \frac{n_b \left(-5.551115123125783^{-17} n_a^3 n_b+2.496 n_a^2 n_b^2+2.7755575615628914^{-17} n_a n_b^3+1.248 n_a^4+1.248 n_b^4\right)}{\left(n_a+n_b\right){}^6} \\
\frac{\partial \alpha_-(n_a,n_b)}{\partial n_b} &=\frac{-8.881784197001252^{-16} n_a^4 n_b-2.496 n_a^3 n_b^2-1.248 n_a n_b^4-1.248 n_a^5}{\left(n_a+n_b\right){}^6}
\end{align}

### $\frac{\partial \tilde{C}}{\partial n}$ Terms:
\begin{align}
\frac{\partial \tilde{c}(n_a,n_b)}{\partial n_a}&=\frac{-0.328565 n_a^6 n_b-0.83592 n_a^5 n_b^2+1.4058 n_a^4 n_b^3-1.73371 n_a^3 n_b^4-3.94548 n_a^2 n_b^5-0.393467 n_a n_b^6-0.0362396 n_a^7-0.131097 n_b^7}{\left(n_a+n_b\right){}^{23/3}}\\
\frac{\partial \tilde{c}(n_a,n_b)}{\partial n_b}&=\frac{0.00842884 n_a^6 n_b-3.33889 n_a^5 n_b^2-0.872507 n_a^4 n_b^3+2.11723 n_a^3 n_b^4-0.319196 n_a^2 n_b^5-0.0564736 n_a n_b^6+0.0760916 n_a^7-0.0187659 n_b^7}{\left(n_a+n_b\right){}^{23/3}}
\end{align}

### $\frac{\partial D}{\partial n}$ Terms:

The results are very very long equations:

\begin{align}
\frac{\partial D}{\partial n_a} &=N_1+n_bN_2 N_3\\
\frac{\partial D}{\partial n_b}&=N_1 - n_aN_2 N_3\\
\end{align}


where:

\begin{align}
N_1&=\frac{3^{2/3} \pi^{4/3} n^{2/3}}
{\sqrt[3]{2}n^7}\left[n_a^7 \left(0.999\, -1.20787 p_a\right)+n_b^7 \left(0.999\, -1.20787 p_b\right)+n_a^6 n_b \left(-5.9992 p_a-0.625467 p_b+4.425\right)+n_a^5 n_b^2 \left(-14.374 p_a-5.0008 p_b+8.139\right)+n_a^4 n_b^3 \left(-29.6827 p_a-13.126 p_b+9.285\right)+n_a^3 n_b^4 \left(-13.126 p_a-29.6827 p_b+9.285\right)+n_a^2 n_b^5 \left(-5.0008 p_a-14.374 p_b+8.139\right)+n_a n_b^6 \left(-0.625467 p_a-5.9992 p_b+4.425\right)\right]\\
N_2 & = \frac{3\ 3^{2/3} \pi ^{4/3} n^{5/3}}{5n^{16} \sqrt[3]{2}} \left[n_a^7 \left(-3.26111 p_a+1.04244 p_b+2.568\right)+n_a^6 n_b \left(-14.5078 p_a+10.6251 p_b+12.84\right)+n_a^5 n_b^2 \left(-3.65133 p_a+32.7073 p_b+23.112\right)+n_a^4 n_b^3 \left(-78.4198 p_a+109.14 p_b+12.84\right)+n_a^3 n_b^4 \left(-109.14 p_a+78.4198 p_b-12.84\right)+n_a^2 n_b^5 \left(-32.7073 p_a+3.65133 p_b-23.112\right)+n_a n_b^6 \left(-10.6251 p_a+14.5078 p_b-12.84\right)+n_b^7 \left(-1.04244 p_a+3.26111 p_b-2.568\right)\right]\\
N_3&=n_a^7 \left(0.999\, -1.20787 p_a\right)+n_b^7 \left(0.999\, -1.20787 p_b\right)+n_a^6 n_b \left(-5.9992 p_a-0.625467 p_b+4.425\right)+n_a^5 n_b^2 \left(-14.374 p_a-5.0008 p_b+8.139\right)+n_a^4 n_b^3 \left(-29.6827 p_a-13.126 p_b+9.285\right)+n_a^3 n_b^4 \left(-13.126 p_a-29.6827 p_b+9.285\right)+n_a^2 n_b^5 \left(-5.0008 p_a-14.374 p_b+8.139\right)+n_a n_b^6 \left(-0.625467 p_a-5.9992 p_b+4.425\right)\\
n &=n_a+n_b\\
p_a &= \left(\frac{n_a}{n_a+n_b}\right){}^{2/3}\\
p_b &= \left(\frac{n_b}{n_a+n_b}\right){}^{2/3}\\
\end{align}
